In [1]:
# TODO:
# test PSQL locally; store as JSON
# date_added, date_visited
# rating_better & rating_worse over time (are the restaurants better/worse after I went?)
# check data before writing, if different throw error
#   - (new categories, historical ratings_history changes, address changes)
# gspread
# unittest

In [2]:
# date_visited_rating = ratings_history[date_visited.year][date_visited.month-1][1]
# most_recent_rating = ratings_history[max(ratings_history.keys())][-1][1]

In [3]:
# Imports
import ast
import bs4
import collections
import copy
import datetime
import json
import pandas
import random
import re
import requests
import sqlalchemy
import sqlite3
import time
import urllib

### Define

In [4]:
def get_yelp_biz_id(soup):
    yelp_biz_id = soup.find('meta', {'name': 'yelp-biz-id'})
    yelp_biz_id = yelp_biz_id['content']
    return yelp_biz_id    # ybid does not need to serialized because it is not a metric_name

In [5]:
def get_yelp_alias(soup):
    # TODO: test that input_yelp_alias == scraped_yelp_alias; url may get re-routed
    yelp_alias = soup.find('meta', {'property': 'og:url'})
    yelp_alias = yelp_alias['content'].split('/')[-1]
    return yelp_alias

In [6]:
def get_is_closed(soup):
    is_closed = 'CLOSED' in soup.title.text
    return is_closed

In [7]:
def get_business_name(soup):
    business_name = soup.find('h1', {'class': 'biz-page-title'})
    business_name = business_name.text.strip()
    return business_name

In [8]:
def get_rating(soup):
    rating = soup.find('div', {'class': 'i-stars'})
    rating = rating['title'].split()[0]
    rating = float(rating)
    return rating

In [9]:
def get_review_count(soup):
    review_count = soup.find('span', {'class': 'review-count rating-qualifier'})
    review_count = review_count.text.strip().split()[0]
    review_count = int(review_count)
    return review_count

In [10]:
def get_ratings_history(soup):
    ratings_history = soup.find('div', {'id': 'rating-details-modal-content'})
    ratings_history = ratings_history['data-monthly-ratings']
    ratings_history = json.loads(ratings_history)
    ratings_history = {int(k):v for k,v in ratings_history.items()}
    return ratings_history

In [11]:
def get_ratings_distribution(soup):
    # Convert HTML table to len-10 string list
    text = soup.find('table', {'class': 'histogram'})
    text = text.text.replace('\n', ' ')
    text = re.sub(' {2,}', ',', text)    # Replace 2+ whitespace with 1 comma
    text = text.strip(',').split(',')
    
    # Convert string list to dictionary
    ratings = [text[i] for i in range(len(text)) if i%2 == 0]
    ratings = [i.split()[0] for i in ratings]
    ratings = [int(i) for i in ratings]
    counts  = [text[i] for i in range(len(text)) if i%2 != 0]
    counts  = [int(i) for i in counts]
    ratings_distribution = {'rating' : ratings, 'count' : counts}
    return ratings_distribution

In [12]:
def get_price_range(soup):
    price_range = soup.find('span', {'class': 'business-attribute price-range'})
    price_range = price_range.text.count('$')
    return price_range

In [13]:
def get_category_list(soup):
    category_list = soup.find('span', {'class': 'category-str-list'})
    category_list = category_list.contents
    category_list = [i.string for i in category_list]
    category_list = [i for i in category_list if '\n' not in i]
    return category_list

In [14]:
def get_address(soup):
    address = soup.find('address')
    address = copy.copy(address)    # Make a copy to prevent modifying the original document
    for tag in address.find_all('br'):
        tag.replace_with('\n')
    address = address.text.strip().split('\n')
    address = ', '.join(address)
    return address

In [15]:
def get_neighborhood_list(soup):
    neighborhood_list = soup.find('span', {'class': 'neighborhood-str-list'})
    if neighborhood_list is not None:
        neighborhood_list = neighborhood_list.text
        neighborhood_list = neighborhood_list.strip().split(', ')
    return neighborhood_list

### Testing

In [ ]:
yelp_alias = 'momofuku-má-pêche-new-york-2'

In [ ]:
urllib.parse.quote_plus(yelp_alias)

In [ ]:
url = 'https://www.yelp.com/biz/{}'.format(yelp_alias)
r = requests.get(url, timeout=5)
soup = bs4.BeautifulSoup(r.text, 'lxml')

In [ ]:
yelp_alias = soup.find('meta', {'property': 'og:url'})
yelp_alias = yelp_alias['content'].split('/')[-1]
yelp_alias

In [ ]:
is_closed = 'CLOSED' in soup.title.text
is_closed

In [ ]:
price_range = soup.find('span', {'class': 'business-attribute price-range'})
price_range = price_range.text.count('$')
price_range

In [ ]:
rating = soup.find('div', {'class': 'i-stars'})
rating = rating['title'].split()[0]
rating = float(rating)
rating

In [ ]:
address = soup.find('address')
address = copy.copy(address)    # Make a copy to prevent modifying the original document
for tag in address.find_all('br'):
    tag.replace_with('\n')
address = address.text.strip().split('\n')
address = ', '.join(address)
address

### Check

In [16]:
# engine = sqlalchemy.create_engine('postgresql://localhost/postgres')
# tables = pandas.read_sql_query("""
# select * from pg_catalog.pg_tables
# where schemaname = 'public'
# and tablename = 'yelp_raw'
# """, engine)
# tables

In [17]:
connection = sqlite3.connect('restaurants.db')
tables = pandas.read_sql_query("""
select * from sqlite_master
where type = 'table'
""", connection)
tables

,type,name,tbl_name,rootpage,sql


In [18]:
if 'yelp_raw' in tables['tbl_name'].values:
    yelp_aliases = yelp_aliases[yelp_aliases['yelp_alias'].notna()]
    yelp_raw = pandas.read_sql_query('select * from yelp_raw', connection)
    yelp_raw['insert_datetime'] = pandas.to_datetime(yelp_raw['insert_datetime'])
    yelp_raw['insert_date'] = yelp_raw['insert_datetime'].dt.date
    count_by_insert_date = yelp_raw.groupby('insert_date').count()['yelp_biz_id']
    count_by_insert_date = count_by_insert_date.rename('count').reset_index()
    count_by_insert_date

### Run

In [19]:
yelp_aliases = pandas.read_csv('yelp_aliases.csv')

In [20]:
# I did not add aliases for some food carts & bakeries; drop them for now
yelp_aliases = yelp_aliases[yelp_aliases['yelp_alias'].notna()]

# I use '-' to represent N/A; convert to None 
yelp_aliases['date_added'] = yelp_aliases['date_added'].apply(lambda x: None if x == '-' else x)
yelp_aliases['date_added'] = pandas.to_datetime(yelp_aliases['date_added'])
yelp_aliases['date_visited'] = pandas.to_datetime(yelp_aliases['date_visited'])

In [21]:
# TODO: proxies & IP pool?
class Connector:
    """ This class handles how to make a connection to a URL. """
    def __init__(self):
        self.attempt_limit = 3    # Attempt to connect N times
        self.sleep = 5            # Sleep N seconds after failing to connect
    
    def make_soup(self, url):
        """ Given a URL, attempt N times to connect.
            If successful, return soup object. """
        # Attempt to connect
        attempt = 1
        while attempt <= self.attempt_limit:
            try:
                r = requests.get(url, timeout=5)
                rand_sleep = random.uniform(1, 3)
                print('Connected, sleeping {} seconds...'.format(round(rand_sleep, 2)))
                time.sleep(rand_sleep)
                break
            except requests.exceptions.ReadTimeout:
                print('Could not connect, sleeping {} seconds...'.format(self.sleep))
                attempt += 1
                time.sleep(self.sleep)
                self.sleep *= 2
        
        # If connected, make soup
        if attempt > self.attempt_limit:
            raise Exception('Failed to connect after {} attempts'.format(self.attempt_limit))
        soup = bs4.BeautifulSoup(r.text, 'lxml')
        return soup

In [22]:
def dict_to_df(data_i):
    """ Convert dictionary to melted DataFrame format.
        data_i represents a single row in pivoted data. """
    # Serialize metric_values (TODO: remove when migrating to PSQL?)
    for key, value in data_i.items():
        data_i[key] = json.dumps(value)
    
    # Melt
    data_i = pandas.Series(data_i).to_frame().T
    data_i = data_i.melt()
    data_i.columns = ['metric_name', 'metric_value']
    data_i.insert(0, 'yelp_biz_id', get_yelp_biz_id(soup))
    data_i.insert(0, 'insert_datetime', datetime.datetime.now())
    return data_i

In [23]:
%%time
# Iterate
seed_data = pandas.DataFrame()    # Seed data is only populated if yelp_raw does not exist
new_data = pandas.DataFrame()
connector = Connector()
for index, row in yelp_aliases.iterrows():
    # Set up
    business_name = row['business_name']
    yelp_alias = row['yelp_alias']
    print([business_name, yelp_alias])
    
    # Attempt to make soup
    url = 'https://www.yelp.com/biz/{}'.format(yelp_alias)
    soup = connector.make_soup(url)
    
    # If yelp_raw does not exist, seed the table with input data
    if 'yelp_raw' not in tables['tbl_name'].values:
        # Get data
        seed_data_i = collections.OrderedDict()
        seed_data_i['business_name'] = business_name
        seed_data_i['yelp_alias'] = yelp_alias
        
        # Format & append
        seed_data_i = dict_to_df(seed_data_i)
        seed_data = seed_data.append(seed_data_i, ignore_index=True)
    
    # Get data
    new_data_i = collections.OrderedDict()
    new_data_i['yelp_alias'] = get_yelp_alias(soup)
    new_data_i['is_closed'] = get_is_closed(soup)
    new_data_i['business_name'] = get_business_name(soup)
    new_data_i['rating'] = get_rating(soup)
    new_data_i['review_count'] = get_review_count(soup)
    new_data_i['ratings_history'] = get_ratings_history(soup)
    new_data_i['price_range'] = get_price_range(soup)
    new_data_i['category_list'] = get_category_list(soup)
    new_data_i['address'] = get_address(soup)
    new_data_i['neighborhood_list'] = get_neighborhood_list(soup)
    
    # Format & append
    new_data_i = dict_to_df(new_data_i)
    new_data = new_data.append(new_data_i, ignore_index=True)

['Muk Eun Ji', 'muk-eun-ji-new-york']
Sleeping 1.97 seconds...
['RamenCo', 'ramenco-new-york-2']
Sleeping 1.75 seconds...
['BLT Bar & Grill', 'blt-bar-and-grill-new-york']
Sleeping 1.95 seconds...
['Bill’s Bar & Burger Downtown', 'bills-bar-and-burger-downtown-new-york']
Sleeping 2.45 seconds...
['Ruchi', 'ruchi-new-york-2']
Sleeping 2.65 seconds...
['Komegashi Too', 'komegashi-too-jersey-city']
Sleeping 1.24 seconds...
['Smashburger', 'smashburger-new-york-6']
Sleeping 1.93 seconds...
["Harry's Italian", 'harrys-italian-new-york-2']
Sleeping 1.67 seconds...
['goa taco', 'goa-taco-new-york']
Sleeping 2.8 seconds...
["Morgenstern's Finest Ice Cream", 'morgensterns-finest-ice-cream-new-york-2']
Sleeping 2.12 seconds...
["Vanessa's Dumpling House", 'vanessas-dumpling-house-new-york-2']
Sleeping 1.88 seconds...
['Philip Marie', 'philip-marie-new-york']
Sleeping 2.21 seconds...
["Joe's Shanghai", 'joes-shanghai-new-york-2']
Sleeping 2.14 seconds...
['Oka Sushi', 'oka-sushi-new-york']
Sleepi

Sleeping 2.25 seconds...
['BRGR', 'brgr-new-york-2']
Sleeping 2.71 seconds...
['Kang Ho Dong Baekjeong', 'kang-ho-dong-baekjeong-new-york']
Sleeping 1.73 seconds...
['Spot Dessert Bar', 'spot-dessert-bar-new-york-2']
Sleeping 2.53 seconds...
['Latham House', 'latham-house-jersey-city']
Sleeping 1.55 seconds...
['Delicatessen', 'delicatessen-new-york']
Sleeping 2.11 seconds...
['The Musket Room', 'the-musket-room-new-york']
Sleeping 2.82 seconds...
['Empellón', 'empellón-new-york-3']
Sleeping 2.28 seconds...
['Rowland’s Bar & Grill', 'rowlands-bar-and-grill-new-york-3']
Sleeping 1.56 seconds...
['Xiao Chuan Dian', 'xiao-chuan-dian-palisades-park-2']
Sleeping 2.35 seconds...
['Sawadee', 'sawadee-jersey-city']
Sleeping 2.29 seconds...
['Churrascaria Plataforma', 'churrascaria-plataforma-new-york-3']
Sleeping 2.57 seconds...
['Noodle Village', 'noodle-village-new-york']
Sleeping 2.66 seconds...
['The Wheeltapper Pub', 'the-wheeltapper-pub-new-york']
Sleeping 2.37 seconds...
['Dons Bogam BB

### new_data vs. latest_data

In [24]:
new_data.head(10)

,insert_datetime,yelp_biz_id,metric_name,metric_value
0,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,yelp_alias,"""muk-eun-ji-new-york"""
1,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,is_closed,true
2,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,business_name,"""Muk Eun Ji"""
3,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,rating,3.5
4,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,review_count,365
5,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,ratings_history,"{""2016"": [[0, 4.0], [1, 3.5], [2, 3.0], [3, 3...."
6,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,price_range,2
7,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,category_list,"[""Korean"", ""Barbeque""]"
8,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,address,"""34 W 32nd St, Fl 1, New York, NY 10001"""
9,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,neighborhood_list,"[""Koreatown"", ""Midtown West""]"


In [25]:
new_data['metric_value'].apply(json.loads).head(10)

0                                  muk-eun-ji-new-york
1                                                 True
2                                           Muk Eun Ji
3                                                  3.5
4                                                  365
5    {'2016': [[0, 4.0], [1, 3.5], [2, 3.0], [3, 3....
6                                                    2
7                                   [Korean, Barbeque]
8               34 W 32nd St, Fl 1, New York, NY 10001
9                            [Koreatown, Midtown West]
Name: metric_value, dtype: object

In [26]:
latest_data = pandas.DataFrame()
if 'yelp_raw' in tables['tbl_name'].values:
    print('(yelp_raw exists, incremental update)')
    latest_data = pandas.read_sql_query("""
    with last_updated as (
        select
        yelp_biz_id, metric_name,
        max(insert_datetime) as last_update_datetime
        from yelp_raw
        group by 1,2 order by 1,2
    )
    select a.*
    from yelp_raw a
    join last_updated b
    on a.yelp_biz_id = b.yelp_biz_id
    and a.metric_name = b.metric_name
    and a.insert_datetime = b.last_update_datetime
    order by 1,2,3
    """, connection)
else:
    print('(continue)')

(continue)


In [27]:
latest_data.head(10)

""


In [28]:
# if latest_data.shape[0] != 0:
#     # Find which metric values changed from the latest data to the newest data
#     # (Only metric values that changed get written to the database)
#     testing = new_data.merge(latest_data, how='outer',
#                              on=['yelp_biz_id', 'metric_name'], suffixes=['_new', '_latest'])
#     testing = testing[testing['metric_value_new'] != testing['metric_value_latest']]
#     testing['metric_value_new'] = testing['metric_value_new'].apply(json.loads)
#     testing['metric_value_latest'] = testing['metric_value_latest'].apply(json.loads)

### Write to DB

In [36]:
if 'yelp_raw' not in tables['tbl_name'].values:
    print('(yelp_raw does not exist, writing seed data)')
    seed_data.to_sql('yelp_raw', connection, if_exists='append', index=False)

(yelp_raw does not exist, writing seed data)


In [37]:
new_data.to_sql('yelp_raw', connection, if_exists='append', index=False)

### Read from DB

In [38]:
data = pandas.read_sql_query("""
with last_updated as (
    select
    yelp_biz_id, metric_name,
    max(insert_datetime) as last_update_datetime
    from yelp_raw
    group by 1,2 order by 1,2
)
select a.*
from yelp_raw a
join last_updated b
on a.yelp_biz_id = b.yelp_biz_id
and a.metric_name = b.metric_name
and a.insert_datetime = b.last_update_datetime
order by 1,2,3
""", connection)

In [41]:
data.head(10)

,insert_datetime,yelp_biz_id,metric_name,metric_value
0,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,address,"""34 W 32nd St, Fl 1, New York, NY 10001"""
1,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,business_name,"""Muk Eun Ji"""
2,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,category_list,"[""Korean"", ""Barbeque""]"
3,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,is_closed,true
4,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,neighborhood_list,"[""Koreatown"", ""Midtown West""]"
5,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,price_range,2
6,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,rating,3.5
7,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,ratings_history,"{""2016"": [[0, 4.0], [1, 3.5], [2, 3.0], [3, 3...."
8,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,review_count,365
9,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,yelp_alias,"""muk-eun-ji-new-york"""


In [42]:
data['metric_value'] = data['metric_value'].apply(json.loads)

In [44]:
data.head(10)

,insert_datetime,yelp_biz_id,metric_name,metric_value
0,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,address,"34 W 32nd St, Fl 1, New York, NY 10001"
1,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,business_name,Muk Eun Ji
2,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,category_list,"[Korean, Barbeque]"
3,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,is_closed,True
4,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,neighborhood_list,"[Koreatown, Midtown West]"
5,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,price_range,2
6,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,rating,3.5
7,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,ratings_history,"{'2016': [[0, 4.0], [1, 3.5], [2, 3.0], [3, 3...."
8,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,review_count,365
9,2018-09-02 09:11:29.028560,g_F9WJJpRFB40oPJdoD2uA,yelp_alias,muk-eun-ji-new-york


In [45]:
data = data.pivot(index='yelp_biz_id', columns='metric_name', values='metric_value')

In [46]:
data.head()

metric_name,address,business_name,category_list,is_closed,neighborhood_list,price_range,rating,ratings_history,review_count,yelp_alias
yelp_biz_id,,,,,,,,,,
-7b17IgC8MNVWDM-JV0-Tw,"9 E 53rd St, New York, NY 10022",Burger Heaven,"[Burgers, Salad, Sandwiches]",False,[Midtown East],2,3,"{'2016': [[0, 2.0], [1, 2.0], [2, 2.0], [3, 2....",205,burger-heaven-new-york
-lgNzAEyFr8LCTuBTWnAMg,"47 W 55th St, New York, NY 10019",Dim Sum Palace,"[Dim Sum, Seafood, Noodles]",False,[Midtown West],2,4,"{'2017': [[6, 4.5], [7, 3.5], [8, 4.0], [9, 3....",157,dim-sum-palace-new-york-7
0087AC_NnFRtC0c9FsB9pw,"17 Waverly Pl, New York, NY 10003",The Boil,"[Cajun/Creole, Seafood]",False,[Greenwich Village],2,4,"{'2016': [[4, 4.5], [5, 4.5], [6, 4.5], [7, 4....",561,the-boil-new-york-5
0CjK3esfpFcxIopebzjFxA,"9 Pell St, New York, NY 10013",Joe’s Shanghai,"[Shanghainese, Seafood, Venues & Event Spaces]",False,"[Chinatown, Civic Center]",2,4,"{'2016': [[0, 4.0], [1, 3.5], [2, 4.0], [3, 3....",5326,joes-shanghai-new-york-2
0LpWRWQx8Agm-mZtsOQ5gg,"40 Wall St, New York, NY 10005",Neapolitan Express,"[Pizza, Italian]",False,[Financial District],2,3.5,"{'2016': [[0, 3.0], [1, 3.0], [2, 3.0], [3, 4....",159,neapolitan-express-new-york-4


In [47]:
data['ratings_history'][0].keys()

dict_keys(['2016', '2017', '2018', '2014', '2015'])

In [48]:
data['is_closed'].unique()

array([False, True], dtype=object)

In [49]:
type(data['is_closed'][0])

bool

### Data Tests

In [ ]:
# TESTS:
# test that ratings_distribution_text is a list of length 10
# test that ratings_distribution data averages to rating

In [ ]:
# ratings_distribution = pandas.DataFrame(ratings_distribution)
# ratings_distribution['contrib'] = ratings_distribution['rating'] * ratings_distribution['count']
# ratings_distribution['contrib'].sum()/ratings_distribution['count'].sum()

In [ ]:
# # TODO: unique_values handling for collections-based columns (i.e. ratings_history)
# unique_values = pandas.read_sql_query("""
#     select * from yelp_raw
#     where metric_name in ('yelp_alias', 'business_name')
#     """, connection) \
#       .groupby(['yelp_biz_id', 'metric_name']) \
#       .agg({'metric_value': ['unique']})
# unique_values.columns = ['metric_value']